# NLP

* Explain how word embeddings capture relationships between words
* Load pre-trained word vectors
* Measure similarity between word vectors using cosine similarity
* Use word embeddings to solve word analogy problems such as Man is to Woman as King is to ______.  
* Modify word embeddings to reduce their **gender bias**

In [1]:
import numpy as np

In [2]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [3]:
words, word_to_vec_map = read_glove_vecs('pretrainedmodel/nlp_glovevec/glove.6B.50d.txt')

In [10]:
list(word_to_vec_map.items())[42]

('which',
 array([ 9.0558e-01,  5.4033e-02, -2.4091e-02,  8.1110e-02,  8.6450e-02,
         6.5504e-01, -3.4224e-01, -7.6129e-01,  1.0258e-01,  5.9494e-02,
         3.0353e-01, -1.0311e-01, -2.8574e-01, -3.5059e-01,  2.3319e-01,
         2.7913e-01, -2.1905e-03,  1.6015e-01, -6.5622e-01, -1.3339e-01,
         3.8494e-01, -2.0867e-01,  2.6137e-01, -9.0254e-02, -3.4935e-01,
        -1.5398e+00, -4.6352e-01,  1.6734e-01, -1.9253e-01, -1.9790e-01,
         4.0080e+00, -2.4514e-01, -1.5461e-01, -2.8890e-01, -4.9511e-02,
        -2.9696e-01,  2.1610e-01, -1.5298e-01, -1.2235e-01,  7.1447e-02,
        -1.1104e-01, -1.5518e-01, -2.6936e-02, -6.7826e-02, -5.6607e-01,
         2.0991e-01, -4.0505e-01, -1.2906e-01, -1.8325e-01, -5.8796e-01]))

## Cosine Similarity

To measure the similarity between two words:

$$\text{CosineSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 ||v||_2} = cos(\theta)$$

* $u \cdot v$ is the dot product (or inner product) of two vectors
* $||u||_2$ is the norm (or length) of the vector $u$
* $\theta$ is the angle between $u$ and $v$. 
* The cosine similarity depends on the angle between $u$ and $v$. 
    * If $u$ and $v$ are very similar, their cosine similarity will be close to 1.
    * If they are dissimilar, the cosine similarity will take a smaller value. 

<img src="images/cosine_sim.png" style="width:800px;height:250px;">
<caption><center><font color='purple'><b>Figure 1</b>: The cosine of the angle between two vectors is a measure of their similarity.</font></center></caption></img>

In [45]:
def cosine_similarity(u, v):
    """
    Cosine similarity between u and v  of shape (n,)
        
    """
    # Special case. Consider the case u = [0, 0], v=[0, 0]
    if np.all(u == v):
        return 1
    
    dot = np.dot(u, v) 
    norm_u = np.linalg.norm(u, ord=2)
    norm_v = np.linalg.norm(v, ord=2)
    
    # Avoid division by 0
    if np.isclose(norm_u * norm_v, 0, atol=1e-32):
        return 0
    
    # Compute the cosine similarity
    cosine_similarity = dot/(norm_u * norm_v)

    return cosine_similarity.round(5)

## Word Analogy

* For example: *man* is to *woman* as *king* is to *queen*
* We're trying to find a word *d*, such that the associated word vectors are related as: $e_b - e_a \approx e_d - e_c$

In [11]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    
    # convert words to lowercase & get word embeddings
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
        
    
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None
    
    for w in word_to_vec_map:   
        
        # avoid best_word being word_c
        if w == word_c:
            continue

        # Compute cosine similarity
        cosine_sim = cosine_similarity(e_b-e_a, word_to_vec_map[w] - e_c)
        
        # Update variables
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
        
    return best_word

In [25]:
triads = [('italy', 'italian', 'spain'), 
          ('india', 'delhi', 'japan'), 
          ('man', 'woman', 'boy'), 
          ('man', 'programmer', 'woman')]

for triad in triads:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad, word_to_vec_map)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
man -> programmer :: woman -> programmer
woman -> homemaker :: man -> mirando


## Debiasing Word Vectors

We examine gender biases that can be reflected in a word embedding, and explore algorithms for reducing the bias. In addition we hone intuition about what word vectors are doing.

First, see how the GloVe word embeddings relate to gender. You'll begin by computing a vector $g = e_{woman}-e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word *woman*, and $e_{man}$ corresponds to the word vector corresponding to the word *man*. The resulting vector $g$ roughly encodes the concept of "gender". 

You might get a more accurate representation if you compute $g_1 = e_{mother}-e_{father}$, $g_2 = e_{girl}-e_{boy}$, etc. and average over them, but just using $e_{woman}-e_{man}$ will give good enough results for now.


In [26]:
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]


In [34]:
print ('List of names and their similarities with constructed vector:')

# girls and boys name
name_list = ['udit', 'kanwal', 'ronak', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 
             'katy', 'yasmin']

for w in name_list:
    print (w, cosine_similarity(word_to_vec_map[w], g).round(3))

List of names and their similarities with constructed vector:
udit -0.005
kanwal 0.114
ronak 0.034
ronaldo -0.312
priya 0.176
rahul -0.169
danielle 0.244
reza -0.079
katy 0.283
yasmin 0.233


Female first names tend to have a positive cosine similarity with our constructed vector $g$, while male first names tend to have a negative cosine similarity. This is not surprising, and the result seems acceptable. 

In [36]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 
             'receptionist', 'technology',  'fashion', 'teacher', 'engineer', 'pilot', 
             'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g).round(3))

Other words and their similarities:
lipstick 0.277
guns -0.189
science -0.061
arts 0.008
literature 0.065
warrior -0.209
doctor 0.119
tree -0.071
receptionist 0.331
technology -0.132
fashion 0.036
teacher 0.179
engineer -0.08
pilot 0.001
computer -0.103
singer 0.185


It is astonishing how these results reflect certain unhealthy gender stereotypes. For example, we see “computer” is negative and is closer in value to male first names, while “literature” is positive and is closer to female first names. Ouch!

Below we'll reduce the bias of these vectors, using an algorithm due to [Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520). Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender-specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. We'll treat these two types of words differently when debiasing.

### Neutralize Bias for Non-Gender Specific Words 

The figure below should helps visualize what neutralizing does. If we're using a 50-dimensional word embedding, the 50 dimensional space can be split into two parts: The bias-direction $g$, and the remaining 49 dimensions, which is called $g_{\perp}$ here. In linear algebra, we say that the 49-dimensional $g_{\perp}$ is perpendicular (or "orthogonal") to $g$, meaning it is at 90 degrees to $g$. The neutralization step takes a vector such as $e_{receptionist}$ and zeros out the component in the direction of $g$, giving us $e_{receptionist}^{debiased}$. 

Even though $g_{\perp}$ is 49-dimensional, given the limitations of what you can draw on a 2D screen, it's illustrated using a 1-dimensional axis below. 

<img src="images/neutral.png" style="width:800px;height:300px;">
<caption><center><font color='purple'><b>Figure 2</b>: The word vector for "receptionist" represented before and after applying the neutralize operation.</font> </center></caption></img>

Given an input embedding $e$, we can use the following formulas to compute $e^{debiased}$, where $e^{bias\_component}$ is the projection of $e$ onto the direction $g$. 

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g$$
$$e^{debiased} = e - e^{bias\_component}$$



In [37]:
def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that gender neutral words are zero in the gender subspace.
    
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """
    
    # Compute e_biascomponent
    e = word_to_vec_map[word]
    e_biascomponent = (np.dot(e, g)* g)/np.square(np.linalg.norm(g, ord=2))
 
    # Neutralize e by subtracting e_biascomponent from it 
    # e_debiased should be equal to its orthogonal projection. 
    e_debiased = e - e_biascomponent
        
    return e_debiased

In [38]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))

cosine similarity between receptionist and g, before neutralizing:  0.3307794175059374
cosine similarity between receptionist and g, after neutralizing:  -2.920516166121757e-17


### Equalization Algorithm for Gender-Specific Words

Let's see how debiasing can be applied to word pairs such as "actress" and "actor" - which we want, should only differ through the gender property.For example, suppose "actress" is closer to "babysit" than "actor." By applying neutralization to "babysit," we can reduce the gender stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. 

**The key idea behind equalization is to make sure that a particular pair of words are equidistant from the 49-dimensional $g_\perp$**. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$, or from any other work that has been neutralized. Visually, this is how equalization works: 

<img src="images/equalize10.png" style="width:800px;height:400px;">


Key equations: 


$$ \mu = \frac{e_{w1} + e_{w2}}{2}\tag{4}$$ 

$$ \mu_{B} = \frac {\mu \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{5}$$ 

$$\mu_{\perp} = \mu - \mu_{B} \tag{6}$$

$$ e_{w1B} = \frac {e_{w1} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{7}$$ 
$$ e_{w2B} = \frac {e_{w2} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{8}$$


$$e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {||(e_{w1} - \mu_{\perp}) - \mu_B||_2} \tag{9}$$


$$e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {||(e_{w2} - \mu_{\perp}) - \mu_B||_2} \tag{10}$$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{11}$$
$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{12}$$

In [42]:
def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    
    # Get word embeddings
    w1, w2 = pair
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]
    
    # Compute the mean of e_w1 and e_w2
    mu = (e_w1 + e_w2)*0.5

    # Compute projection of mu over the bias axis and the orthogonal axis
    mu_B = (np.dot(mu, bias_axis) * bias_axis) / np.square(np.linalg.norm(bias_axis, ord=2))
    mu_orth = mu - mu_B

    # Projection of words on bias_axis
    e_w1B = (np.dot(e_w1, bias_axis)* bias_axis)/ np.square(np.linalg.norm(bias_axis, ord=2)) 
    e_w2B = (np.dot(e_w2, bias_axis)* bias_axis)/ np.square(np.linalg.norm(bias_axis, ord=2)) 
    
    # Adjust the Bias part of e_w1B and e_w2B (refer formulas above)
    corrected_e_w1B = np.sqrt(np.abs(1 - np.square(np.linalg.norm(mu_orth,2)))) * (e_w1B - mu_B) / np.linalg.norm((e_w1 - mu_orth - mu_B), ord=2)
    corrected_e_w2B = np.sqrt(np.abs(1 - np.square(np.linalg.norm(mu_orth,2)))) * (e_w2B - mu_B) / np.linalg.norm((e_w2 - mu_orth - mu_B), ord=2)

    # Debias by equalizing e1 and e2 to the sum of their corrected projections
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth

    return e1, e2

In [46]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()

e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.11711
cosine_similarity(word_to_vec_map["woman"], gender) =  0.35667

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.70044
cosine_similarity(e2, gender) =  0.70044


In [47]:
print("cosine similarities before equalizing:")
print("cosine_similarity(\"actor\", gender) = ", cosine_similarity(word_to_vec_map["actor"], g))
print("cosine_similarity(\"actoress\", gender) = ", cosine_similarity(word_to_vec_map["actress"], g))
print()

e1, e2 = equalize(("actor", "actress"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity("actor", gender) =  -0.08388
cosine_similarity("actoress", gender) =  0.33422

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.60833
cosine_similarity(e2, gender) =  0.60833
